<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [ ]:
import sys
from pathlib import Path

src_path = Path("..")
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

from app.src.agent.utils.infos import list_snake_models
from app.src.agent.training.train import train_snake

<h3 style="color:#CCC229">SOME TESTS<h3>

In [ ]:
# Example of listing all available Snake models
list_snake_models()

In [ ]:
# Example of training a Snake agent using PPO algorithm
train_snake(timesteps=1_000_000, grid_size=5, n_envs=10, algorithm="PPO")

In [ ]:
# Example of resuming training from a previous model using its UUID
train_snake(timesteps=2_000_000, n_envs=10, algorithm="PPO", base_uuid="ddfab7de-d0f0-4e44-a59b-0ac0604aba0d")